# 소스 코드 설명

이 파일은 Azure Cosmos DB의 Semantic Cache 기능을 설명하고 사용하는 Jupyter Notebook입니다. 주요 기능은 다음과 같습니다:
- Azure Cosmos DB 연결 설정
- 데이터베이스 및 컨테이너 생성
- 데이터 삽입 및 쿼리
- Semantic Cache 설정 및 사용

아래에 소스 코드가 있습니다.

In [1]:
import os
import pymongo
from dotenv import load_dotenv
from pymongo import MongoClient
from langchain_community.cache import AzureCosmosDBSemanticCache
from langchain.globals import set_llm_cache
from langchain_community.vectorstores.azure_cosmos_db import (
    CosmosDBSimilarityType,
    CosmosDBVectorSearchType,
)
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

# .env 파일 로드
load_dotenv()

# Azure Cosmos DB 및 Azure OpenAI 연결 설정
NAMESPACE = os.getenv('DB_NAMESPACE')
CONNECTION_STRING = (os.getenv('DB_CONNECTION_STRING'))
DB_NAME, COLLECTION_NAME = NAMESPACE.split(".")
API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
CHAT_MODEL_DEPLOY_NAME = os.getenv('AZURE_OPENAI_CHAT_MODEL_DEPLOY_NAME')
EMBEDDINGS_MODEL_DEPLOY_NAME = os.getenv('AZURE_OPENAI_EMBEDDINGS_MODEL_DEPLOY_NAME')

# Cosmos DB를 위한 MongoDB 클라이언트 초기화
mongo_client = MongoClient(CONNECTION_STRING)

# 차원 설정 (임베딩 모델에 맞게 설정)
DIMENSIONS = 1536  # 예시 값, 모델에 따라 조정 필요

def init_llm_chain():
    # 기존 캐시가 있으면 삭제
    mongo_client[DB_NAME][COLLECTION_NAME].drop_indexes()
    mongo_client[DB_NAME].drop_collection(COLLECTION_NAME)

    # LLM 프롬프트를 위한 템플릿 정의
    prompt_template = """
    당신은 질문에 답변하는 것을 매우 기뻐하는 AI 어시스턴트입니다.

    질문: {question}
    답을 모르면 모른다고 말하세요. 답을 지어내려고 하지 마세요.
    """
    chatbot_prompt = PromptTemplate(
        template=prompt_template, input_variables=["question", "context"])

    # 모델 버전 0301 이상 필요
    # Azure OpenAI에 배포된 완성 모델 지정
    llm = AzureChatOpenAI(
        deployment_name=CHAT_MODEL_NAME,
        model_name="gpt-35-turbo",
        api_key=API_KEY,
        azure_endpoint=ENDPOINT,
        api_version="2024-05-01-preview",
        cache=True,
        n=1)

    # Azure OpenAI에 배포된 임베딩 모델 지정
    embeddings = AzureOpenAIEmbeddings(
        azure_deployment=EMBEDDINGS_MODEL_NAME,
        model="text-embedding-ada-002",
        api_key=API_KEY,
        azure_endpoint=ENDPOINT,

    # 간단한 LLM 체인 설정
    llm_chain = LLMChain(llm=llm, prompt=chatbot_prompt)

    # LLM을 위한 시맨틱 캐시 설정
    num_lists = 1
    similarity_algorithm = CosmosDBSimilarityType.COS
    kind = CosmosDBVectorSearchType.VECTOR_IVF

    score_threshold = 0.9

    # LLM을 위한 시맨틱 캐시 설정
    set_llm_cache(
        AzureCosmosDBSemanticCache(
            cosmosdb_connection_string=CONNECTION_STRING,
            cosmosdb_client=mongo_client,
            embedding=embeddings,
            database_name=DB_NAME,
            collection_name=COLLECTION_NAME,
            num_lists=num_lists,
            similarity=similarity_algorithm,
            kind=kind,
            score_threshold=score_threshold
        )
    )

    return llm_chain

# llm 체인 초기화
llm_chain = init_llm_chain()

C:\Users\quiett20\AppData\Local\Temp\ipykernel_6400\2273003635.py:21: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  mongo_client = MongoClient(CONNECTION_STRING)
C:\Users\quiett20\AppData\Local\Temp\ipykernel_6400\2273003635.py:60: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=chatbot_prompt)


In [2]:
%%time
# 두 번째 실행: 질문/응답이 이제 Cosmos DB에 캐시되었으므로 검색 속도가 빨라질 것입니다.
llm_chain.invoke("Tell me something interesting about beer making")

CPU times: total: 156 ms
Wall time: 7.41 s


{'question': 'Tell me something interesting about beer making',
 'text': 'Did you know that the ancient Egyptians were actually some of the first brewers in history? They brewed beer for religious ceremonies and daily consumption, using techniques that have been passed down through the ages. Even today, many of the fundamental steps in beer making remain the same as they were thousands of years ago.'}

In [3]:
%%time
# This quesiton/response is not yet cachced in Cosmos DB, so retrieval should be slower
llm_chain("Tell me a joke about tomatoes and food.")

<timed eval>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.


CPU times: total: 0 ns
Wall time: 450 ms


{'question': 'Tell me a joke about tomatoes and food.',
 'text': 'Did you know that the ancient Egyptians were actually some of the first brewers in history? They brewed beer for religious ceremonies and daily consumption, using techniques that have been passed down through the ages. Even today, many of the fundamental steps in beer making remain the same as they were thousands of years ago.'}